# STOCK PREDICTION OF TESLA COMPANY USING LSTM

Before we train the model we need to first get the packages and we also need to download the dataset
In this Analysis I used dataset of tesla company which I downloaded from kaggle.
We will then import the necessary packages we need for reading dataset and other for visualization.

In [10]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
data=pd.read_csv("TSLA.csv")
data.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-07-22,276.220001,280.786682,270.713318,272.243347,272.243347,103472700
1,2022-07-25,272.216675,274.146667,267.399994,268.433319,268.433319,64073400
2,2022-07-26,266.513336,267.309998,256.263336,258.859985,258.859985,66820800
3,2022-07-27,263.809998,275.926666,261.790009,274.820007,274.820007,88110000
4,2022-07-28,280.066681,283.299988,272.799988,280.899994,280.899994,84723000
5,2022-07-29,280.700012,298.320007,279.100006,297.149994,297.149994,95313000
6,2022-08-01,301.276672,311.876678,295.000000,297.276672,297.276672,117042900
7,2022-08-02,294.003326,307.833344,292.666656,300.586670,300.586670,95577600
8,2022-08-03,305.000000,309.549988,301.149994,307.396667,307.396667,80091000
9,2022-08-04,311.000000,313.606659,305.000000,308.633331,308.633331,72256200


We will get the detailed information about the dataset.

In [61]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 251 entries, 0 to 250
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       251 non-null    object 
 1   Open       251 non-null    float64
 2   High       251 non-null    float64
 3   Low        251 non-null    float64
 4   Close      251 non-null    float64
 5   Adj Close  251 non-null    float64
 6   Volume     251 non-null    int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 13.9+ KB


Now we will get a graphical analysis about the stock of tesla.

In [60]:
import plotly.graph_objects as go
figure = go.Figure(data=[go.Candlestick(x=data["Date"],
                                        open=data["Open"], 
                                        high=data["High"],
                                        low=data["Low"], 
                                        close=data["Close"])])
figure.update_layout(title = "TESLA Stock Price Analysis", 
                     xaxis_rangeslider_visible=False)
figure.show()

Now we should decide the dependent and independent features for our dataset. So we find the correlation between them.

In [29]:
correlation=data.corr()
correlation['Close']

Open         0.992804
High         0.996870
Low          0.997570
Close        1.000000
Adj Close    1.000000
Volume      -0.534982
Name: Close, dtype: float64

From the correlation analysis we understand the dependency of the featues . 
Here we will consider Open,High and Low as the independent features and Close as the dependent features.
We will then convert them into array because here y(i.e; the dependent dataset) has only one column which LSTM cannot take.
So we will resize y to (-1,1) due to which we even need to convert x into array form maintaing homogenity in the datasets.

In [30]:
x=data.iloc[:,1:4]
x=x.to_numpy()

array([[276.220001, 280.786682, 270.713318],
       [272.216675, 274.146667, 267.399994],
       [266.513336, 267.309998, 256.263336],
       [263.809998, 275.926666, 261.790009],
       [280.066681, 283.299988, 272.799988],
       [280.700012, 298.320007, 279.100006],
       [301.276672, 311.876678, 295.      ],
       [294.003326, 307.833344, 292.666656],
       [305.      , 309.549988, 301.149994],
       [311.      , 313.606659, 305.      ],
       [302.670013, 304.606659, 285.543335],
       [295.      , 305.200012, 289.08667 ],
       [290.293335, 292.396667, 279.353333],
       [297.066681, 297.51001 , 283.369995],
       [296.513336, 298.236664, 285.833344],
       [289.416656, 300.160004, 285.033325],
       [301.786682, 313.133331, 301.230011],
       [311.666656, 314.666656, 302.883331],
       [303.396667, 309.656677, 300.033325],
       [306.      , 306.5     , 301.853333],
       [299.      , 300.359985, 292.5     ],
       [291.91333 , 292.399994, 286.296661],
       [29

In [31]:
y=data.iloc[:,4]
y=y.to_numpy().reshape(1,-1)
y

array([[272.243347, 268.433319, 258.859985, 274.820007, 280.899994,
        297.149994, 297.276672, 300.58667 , 307.396667, 308.633331,
        288.170013, 290.42334 , 283.333344, 294.356659, 286.630005,
        300.029999, 309.320007, 306.563324, 303.996674, 302.869995,
        296.666656, 289.91333 , 296.453339, 297.09668 , 296.070007,
        288.089996, 284.820007, 277.700012, 275.609985, 277.160004,
        270.209991, 274.420013, 283.700012, 289.26001 , 299.679993,
        304.420013, 292.130005, 302.609985, 303.75    , 303.350006,
        309.070007, 308.730011, 300.799988, 288.589996, 275.329987,
        276.01001 , 282.940002, 287.809998, 268.209991, 265.25    ,
        242.399994, 249.440002, 240.809998, 238.130005, 223.070007,
        222.960007, 216.5     , 217.240005, 221.720001, 204.990005,
        219.350006, 220.190002, 222.039993, 207.279999, 214.440002,
        211.25    , 222.419998, 224.639999, 225.089996, 228.520004,
        227.539993, 227.820007, 214.979996, 215.

Now we will preprocess the data so that model can be developed easily.

In [15]:
sc=MinMaxScaler(feature_range=(0,1))
x = sc.fit_transform(train)
x

array([[0.83012784, 0.83303503, 0.82889203],
       [0.81094257, 0.80031216, 0.81263191],
       [0.78361028, 0.76662015, 0.75797883],
       [0.77065498, 0.80908423, 0.78510095],
       [0.84856241, 0.84542093, 0.83913235],
       [0.85159755, 0.91944156, 0.87004965],
       [0.95020774, 0.98625062, 0.94807878],
       [0.91535145, 0.96632454, 0.93662791],
       [0.96805117, 0.97478439, 0.97825984],
       [0.99680516, 0.9947762 , 0.99715372],
       [0.9568851 , 0.95042301, 0.90167025],
       [0.92012784, 0.95334713, 0.91905915],
       [0.89757194, 0.89025056, 0.87129285],
       [0.93003207, 0.91544979, 0.8910046 ],
       [0.92738025, 0.91903084, 0.90309347],
       [0.8933706 , 0.92850931, 0.89916738],
       [0.95265188, 0.99244357, 0.97865252],
       [1.        , 1.        , 0.98676618],
       [0.96036746, 0.97531016, 0.97277979],
       [0.9728435 , 0.95975364, 0.98171147],
       [0.93929717, 0.92949484, 0.93581004],
       [0.90533549, 0.89026696, 0.90536719],
       [0.

In [32]:
y=target.to_numpy().reshape(-1,1)
y=sc.fit_transform(y)
y

array([[0.81574069],
       [0.79680605],
       [0.7492296 ],
       [0.82854588],
       [0.8587615 ],
       [0.93951887],
       [0.94014842],
       [0.95659807],
       [0.99044161],
       [0.99658744],
       [0.8948912 ],
       [0.90608952],
       [0.87085448],
       [0.92563688],
       [0.88723784],
       [0.95383159],
       [1.        ],
       [0.98630015],
       [0.97354471],
       [0.96794547],
       [0.93711684],
       [0.90355493],
       [0.93605672],
       [0.93925392],
       [0.93415168],
       [0.89449354],
       [0.87824272],
       [0.84285859],
       [0.83247182],
       [0.84017493],
       [0.80563555],
       [0.82655803],
       [0.8726767 ],
       [0.90030814],
       [0.95209217],
       [0.97564857],
       [0.91457111],
       [0.96665331],
       [0.97231882],
       [0.97033098],
       [0.99875758],
       [0.99706791],
       [0.95765819],
       [0.89697838],
       [0.83108032],
       [0.83445982],
       [0.86889969],
       [0.893

Now we will split the dataset for training and testing seperately.

In [37]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y,random_state=35)
xtrain

,Open,High,Low
81,195.880005,200.820007,192.059998
28,280.619995,281.250000,271.809998
221,224.220001,235.229996,223.009995
159,175.130005,178.289993,168.440002
135,183.949997,199.000000,183.690002
...,...,...,...
33,281.299988,289.500000,279.760010
183,183.949997,186.279999,182.009995
236,260.600006,264.450012,259.890015
15,289.416656,300.160004,285.033325


In [35]:
xtest

,Open,High,Low
77,190.779999,195.889999,177.119995
169,191.649994,192.360001,187.149994
137,196.429993,197.500000,189.550003
25,297.429993,302.000000,287.470001
92,197.080002,198.919998,191.800003
...,...,...,...
225,260.170013,261.570007,250.500000
43,299.859985,301.290009,285.820007
97,172.199997,175.199997,169.059998
83,183.960007,186.160004,180.899994


In [38]:
ytrain

array([[0.42898318],
       [0.83247182],
       [0.62995725],
       [0.32471922],
       [0.40691777],
       [0.3717324 ],
       [0.78098596],
       [0.53115989],
       [0.89310204],
       [0.35826459],
       [0.00496968],
       [0.95659807],
       [0.90355493],
       [0.91457111],
       [0.86889969],
       [0.35259915],
       [0.31651919],
       [0.        ],
       [0.82655803],
       [0.84385244],
       [0.40299175],
       [0.24202367],
       [0.41273229],
       [0.36000393],
       [0.42277108],
       [0.35796642],
       [0.41804989],
       [0.42595168],
       [0.14645665],
       [0.49463275],
       [0.75499445],
       [0.80563555],
       [0.01113208],
       [0.05799622],
       [0.43007652],
       [0.92563688],
       [0.34688397],
       [0.56465559],
       [0.94014842],
       [0.99875758],
       [0.44468741],
       [0.76930717],
       [0.43067286],
       [0.36725971],
       [0.95765819],
       [0.28317265],
       [0.25926848],
       [0.557

In [39]:
ytest

array([[0.34534338],
       [0.40905478],
       [0.44086073],
       [0.89449354],
       [0.43037469],
       [0.52847629],
       [0.98630015],
       [0.4307226 ],
       [0.73620905],
       [0.37183181],
       [0.17786502],
       [0.82854588],
       [0.37322332],
       [0.46247889],
       [0.41059536],
       [0.75757879],
       [0.81574069],
       [0.41148987],
       [0.49378792],
       [0.97231882],
       [0.56813435],
       [0.4980121 ],
       [0.37873967],
       [0.36432762],
       [0.07106651],
       [0.44125831],
       [0.46312489],
       [0.36949605],
       [0.49383758],
       [0.07494286],
       [0.41983898],
       [0.37153364],
       [0.8948912 ],
       [0.52748237],
       [0.5814034 ],
       [0.93605672],
       [0.26702118],
       [0.20758371],
       [0.88723784],
       [0.09477189],
       [0.87085448],
       [0.84285859],
       [0.32680648],
       [0.59357912],
       [0.25946724],
       [0.40140142],
       [0.24634727],
       [0.594

Here we will Desig the architecture of LSTM neural network and in final we get the summary of the model.

In [44]:
from keras.models import Sequential
from keras.layers import Dense,LSTM
model=Sequential()
model.add(LSTM(128,return_sequences=True, input_shape=(xtrain.shape[1],1) ))
model.add(LSTM(64, return_sequences=False))
model.add(Dense(25))
model.add(Dense(1))
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 3, 128)            66560     
                                                                 
 lstm_2 (LSTM)               (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 25)                1625      
                                                                 
 dense_1 (Dense)             (None, 1)                 26        
                                                                 
Total params: 117619 (459.45 KB)
Trainable params: 117619 (459.45 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


We will compile the model and then pass the dataset over the model.

In [52]:
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(xtrain,ytrain,batch_size=1, epochs=30)

Epoch 1/30
188/188 [==============================] - 3s 3ms/step - loss: 0.0026
Epoch 2/30
188/188 [==============================] - 1s 3ms/step - loss: 0.0014
Epoch 3/30
188/188 [==============================] - 1s 3ms/step - loss: 0.0035
Epoch 4/30
188/188 [==============================] - 1s 3ms/step - loss: 0.0013
Epoch 5/30
188/188 [==============================] - 1s 3ms/step - loss: 0.0014
Epoch 6/30
188/188 [==============================] - 1s 3ms/step - loss: 0.0013
Epoch 7/30
188/188 [==============================] - 1s 3ms/step - loss: 0.0012
Epoch 8/30
188/188 [==============================] - 1s 3ms/step - loss: 0.0014
Epoch 9/30
188/188 [==============================] - 1s 3ms/step - loss: 0.0010
Epoch 10/30
188/188 [==============================] - 1s 3ms/step - loss: 0.0015
Epoch 11/30
188/188 [==============================] - 1s 3ms/step - loss: 0.0011
Epoch 12/30
188/188 [==============================] - 1s 3ms/step - loss: 0.0012
Epoch 13/30
188/188 [====

We will now predict the model on the test datasets.

In [50]:
from sklearn.metrics import accuracy_score, confusion_matrix
pred=model.predict(xtest)
predicted= sc.inverse_transform(pred)
predicted

2/2 [==============================] - 0s 3ms/step


array([[197.22409],
       [203.88948],
       [207.11903],
       [303.8332 ],
       [209.20178],
       [222.85745],
       [312.63605],
       [210.53003],
       [271.2521 ],
       [196.56563],
       [159.1593 ],
       [283.96146],
       [190.52957],
       [215.11693],
       [198.24767],
       [278.2742 ],
       [290.85632],
       [203.2172 ],
       [215.6036 ],
       [311.02383],
       [230.41403],
       [216.08292],
       [199.95549],
       [188.75897],
       [133.42802],
       [211.98445],
       [212.11511],
       [198.67856],
       [224.87842],
       [137.09612],
       [208.03778],
       [197.13788],
       [303.18262],
       [224.87863],
       [243.93452],
       [303.59293],
       [176.0434 ],
       [164.96085],
       [302.4853 ],
       [142.27284],
       [298.04617],
       [293.45892],
       [184.53154],
       [242.08995],
       [176.56212],
       [197.59285],
       [171.3199 ],
       [248.57076],
       [192.83786],
       [194.10141],


Finally to test our model we will pass the features to the model.

In [58]:
import numpy as np
#features = [Open, High, Low, Adj Close, Volume]
features = np.array([[177.089996, 180.419998, 177.070007]])
res=model.predict(features)
res[0][0]*1000

1/1 [==============================] - 0s 15ms/step


367.79651045799255

# SUMMARY

In final we made a model for stock prediction of tesla company using LSTM Neural Networks.